In [1]:
import os
import glob
import re
import time
import pandas as pd
import requests
from difflib import SequenceMatcher

In [6]:
triplet_files = glob.glob("*/Triplets/**/*.csv", recursive=True)

mapping_records = []

for file in triplet_files:

    df = pd.read_csv(file)

    # Identify split
    if "SKG" in file:
        split = "SKG"
        stanza_split = "SKG_Papers"
    elif "Novel_Papers" in file:
        split = "NOVEL"
        stanza_split = "Novel_Papers"
    elif "Blogs" in file:
        split = "BLOG"
        stanza_split = "Blogs"
    else:
        continue

    filename = os.path.basename(file)
    domain_raw = filename.replace("_triplets.csv", "")

    # BLOG domain cleanup only
    if split == "BLOG":
        domain = domain_raw.replace("_Blogs", "")
    else:
        domain = domain_raw

    paper_ids = sorted(df["paper_ID"].unique())

    pattern = f"*/stanza_files/{stanza_split}/{domain}/*-Stanza-out.txt"
    directory = glob.glob(pattern)

    for pid in paper_ids:
        if pid < len(directory):
            filepath = directory[pid]
            fname = os.path.basename(filepath)

            title = fname.replace("-Stanza-out.txt", "")
            title = re.sub(r"^\d+\.", "", title).strip()

            mapping_records.append({
                "split": split,
                "domain": domain,
                "paper_ID": pid,
                "title": title
            })

paper_mapping = pd.DataFrame(mapping_records)

print("Columns:", paper_mapping.columns)
print("Split distribution:")
print(paper_mapping["split"].value_counts())

Columns: Index(['split', 'domain', 'paper_ID', 'title'], dtype='object')
Split distribution:
SKG      3279
NOVEL     601
BLOG       84
Name: split, dtype: int64


In [7]:
paper_mapping = paper_mapping[
    paper_mapping["split"].isin(["SKG", "NOVEL"])
].reset_index(drop=True)

print("Total scholarly papers:", len(paper_mapping))

Total scholarly papers: 3880


In [8]:
session = requests.Session()
session.headers.update({
    "User-Agent": "ScientificNoveltyProject/1.0 (your_email@gmail.com)"
})

In [9]:
def similar(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def get_openalex_metadata(title):

    url = "https://api.openalex.org/works"

    params = {
        "filter": f'title.search:"{title}"',
        "mailto": "your_email@gmail.com"
    }

    try:
        r = session.get(url, params=params, timeout=15)

        if r.status_code == 200:
            results = r.json()["results"]

            if len(results) > 0:
                best = results[0]

                if similar(title, best["display_name"]) > 0.85:
                    return {
                        "openalex_id": best.get("id"),
                        "publication_year": best.get("publication_year"),
                        "citation_count": best.get("cited_by_count"),
                        "referenced_works": best.get("referenced_works")
                    }
    except:
        pass

    return None

In [10]:
metadata_records = []

for i, row in paper_mapping.iterrows():

    meta = get_openalex_metadata(row["title"])

    if meta:
        metadata_records.append({
            **row,
            **meta
        })

    if i % 50 == 0:
        print("Processed:", i)
        pd.DataFrame(metadata_records).to_csv("partial_metadata.csv", index=False)

    time.sleep(0.4)  # Safe delay

paper_metadata = pd.DataFrame(metadata_records)

print("Matched papers:", len(paper_metadata))

Processed: 0
Processed: 50
Processed: 100
Processed: 150
Processed: 200
Processed: 250
Processed: 300
Processed: 350
Processed: 400
Processed: 450
Processed: 500
Processed: 550
Processed: 600
Processed: 650
Processed: 700
Processed: 750
Processed: 800
Processed: 850
Processed: 900
Processed: 950
Processed: 1000
Processed: 1050
Processed: 1100
Processed: 1150
Processed: 1200
Processed: 1250
Processed: 1300
Processed: 1350
Processed: 1400
Processed: 1450
Processed: 1500
Processed: 1550
Processed: 1600
Processed: 1650
Processed: 1700
Processed: 1750
Processed: 1800
Processed: 1850
Processed: 1900
Processed: 1950
Processed: 2000
Processed: 2050
Processed: 2100
Processed: 2150
Processed: 2200
Processed: 2250
Processed: 2300
Processed: 2350
Processed: 2400
Processed: 2450
Processed: 2500
Processed: 2550
Processed: 2600
Processed: 2650
Processed: 2700
Processed: 2750
Processed: 2800
Processed: 2850
Processed: 2900
Processed: 2950
Processed: 3000
Processed: 3050
Processed: 3100
Processed: 3150

In [17]:
def get_openalex_metadata(title):

    url = "https://api.openalex.org/works"

    params = {
        "search": title,
        "mailto": "your_email@gmail.com"
    }

    for attempt in range(5):
        try:
            r = session.get(url, params=params, timeout=15)

            if r.status_code == 200:
                results = r.json()["results"]

                if len(results) > 0:
                    best = results[0]

                    if similar(title, best["display_name"]) > 0.75:
                        return {
                            "openalex_id": best.get("id"),
                            "publication_year": best.get("publication_year"),
                            "citation_count": best.get("cited_by_count"),
                            "referenced_works": best.get("referenced_works")
                        }
                return None

            elif r.status_code == 429:
                wait = 2 ** attempt
                print(f"Rate limited. Sleeping {wait}s")
                time.sleep(wait)

            else:
                return None

        except Exception as e:
            wait = 2 ** attempt
            print(f"Error: {e}. Sleeping {wait}s")
            time.sleep(wait)

    return None

In [18]:
sample = paper_mapping.head(5)

for t in sample["title"]:
    print("TITLE:", t)
    print(get_openalex_metadata(t))
    print("------")

TITLE: Preview, Attend and Review Schema-Aware Curriculum Learning for Multi-Domain Dialogue State Tracking
Rate limited. Sleeping 1s
Rate limited. Sleeping 2s
Rate limited. Sleeping 4s
Rate limited. Sleeping 8s
Rate limited. Sleeping 16s
None
------
TITLE: Constructing Multi-Modal Dialogue Dataset by Replacing Text with Semantically Relevant Images
Rate limited. Sleeping 1s
Rate limited. Sleeping 2s
Rate limited. Sleeping 4s
Rate limited. Sleeping 8s
Rate limited. Sleeping 16s
None
------
TITLE: Fine-grained Post-training for Improving Retrieval-based Dialogue Systems
Rate limited. Sleeping 1s
Rate limited. Sleeping 2s
Rate limited. Sleeping 4s
Rate limited. Sleeping 8s
Rate limited. Sleeping 16s
None
------
TITLE: Adding Chit-Chat to Enhance Task-Oriented Dialogues
Rate limited. Sleeping 1s
Rate limited. Sleeping 2s
Rate limited. Sleeping 4s
Rate limited. Sleeping 8s
Rate limited. Sleeping 16s
None
------
TITLE: Saying No is An Art Contextualized Fallback Responses for Unanswerable D

In [14]:
def debug_openalex(title):
    url = "https://api.openalex.org/works"
    params = {
        "search": title,
        "mailto": "your_email@gmail.com"
    }

    r = session.get(url, params=params)
    if r.status_code == 200:
        results = r.json()["results"]
        if len(results) > 0:
            for i, res in enumerate(results[:3]):
                print("Result", i)
                print("Title:", res["display_name"])
                print("Year:", res["publication_year"])
                print("ID:", res["id"])
                print("------")
        else:
            print("No results")
    else:
        print("Status:", r.status_code)

In [15]:
debug_openalex("Constructing Multi-Modal Dialogue Dataset by Replacing Text with Semantically Relevant Images")

Status: 429


In [16]:
def get_openalex_metadata(title):

    url = "https://api.openalex.org/works"

    params = {
        "search": title,
        "mailto": "your_email@gmail.com"
    }

    for attempt in range(5):
        try:
            r = session.get(url, params=params, timeout=15)

            if r.status_code == 200:
                results = r.json()["results"]

                if len(results) > 0:
                    best = results[0]

                    if similar(title, best["display_name"]) > 0.75:
                        return {
                            "openalex_id": best.get("id"),
                            "publication_year": best.get("publication_year"),
                            "citation_count": best.get("cited_by_count"),
                            "referenced_works": best.get("referenced_works")
                        }
                return None

            elif r.status_code == 429:
                wait = 2 ** attempt
                print(f"Rate limited. Sleeping {wait}s")
                time.sleep(wait)

            else:
                return None

        except Exception as e:
            wait = 2 ** attempt
            print(f"Error: {e}. Sleeping {wait}s")
            time.sleep(wait)

    return None

In [ ]:
import os
import glob
import re
import time
import pandas as pd
import requests
from difflib import SequenceMatcher
from collections import defaultdict
import hashlib
import json

# ============================================
# PART 1: Load SciND Papers and Extract Titles
# ============================================

print("📚 Loading SciND papers...")
triplet_files = glob.glob("*/Triplets/**/*.csv", recursive=True)

mapping_records = []

for file in triplet_files:
    df = pd.read_csv(file)

    # Identify split
    if "SKG" in file:
        split = "SKG"
        stanza_split = "SKG_Papers"
    elif "Novel_Papers" in file:
        split = "NOVEL"
        stanza_split = "Novel_Papers"
    elif "Blogs" in file:
        split = "BLOG"
        stanza_split = "Blogs"
    else:
        continue

    filename = os.path.basename(file)
    domain_raw = filename.replace("_triplets.csv", "")

    # BLOG domain cleanup
    if split == "BLOG":
        domain = domain_raw.replace("_Blogs", "")
    else:
        domain = domain_raw

    paper_ids = sorted(df["paper_ID"].unique())

    pattern = f"*/stanza_files/{stanza_split}/{domain}/*-Stanza-out.txt"
    directory = glob.glob(pattern)

    for pid in paper_ids:
        if pid < len(directory):
            filepath = directory[pid]
            fname = os.path.basename(filepath)

            title = fname.replace("-Stanza-out.txt", "")
            title = re.sub(r"^\d+\.", "", title).strip()

            mapping_records.append({
                "split": split,
                "domain": domain,
                "paper_ID": pid,
                "title": title
            })

paper_mapping = pd.DataFrame(mapping_records)
print(f"✅ Loaded {len(paper_mapping)} papers")
print(f"   Split distribution:\n{paper_mapping['split'].value_counts()}")

# Keep only scholarly papers (SKG and NOVEL)
paper_mapping = paper_mapping[
    paper_mapping["split"].isin(["SKG", "NOVEL"])
].reset_index(drop=True)
print(f"✅ Keeping {len(paper_mapping)} scholarly papers")

# ============================================
# PART 2: Smart OpenAlex API Client
# ============================================

class OpenAlexClient:
    def __init__(self, email="your_email@gmail.com", delay=0.1):
        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": f"ScientificNoveltyProject/1.0 ({email})"
        })
        self.email = email
        self.delay = delay
        self.last_request = 0
        self.stats = {
            "total": 0,
            "success": 0,
            "rate_limited": 0,
            "not_found": 0
        }

    def _wait_for_rate_limit(self):
        """Ensure we don't exceed 10 requests per second"""
        elapsed = time.time() - self.last_request
        if elapsed < self.delay:
            time.sleep(self.delay - elapsed)

    def normalize_title(self, title):
        """Normalize title for better matching"""
        # Remove special characters, extra spaces
        title = re.sub(r'[^\w\s]', ' ', title.lower())
        title = re.sub(r'\s+', ' ', title).strip()
        return title

    def similarity(self, a, b):
        """Compute title similarity"""
        return SequenceMatcher(None, a.lower(), b.lower()).ratio()

    def search_paper(self, title, year=None):
        """
        Search for a paper in OpenAlex with multiple strategies
        """
        self.stats["total"] += 1
        self._wait_for_rate_limit()

        # Strategy 1: Exact title search with filter
        params = {
            "filter": f'title.search:"{title}"',
            "sort": "relevance_score:desc",
            "per-page": 5,
            "mailto": self.email
        }
        if year:
            params["filter"] += f",publication_year:{year}"

        try:
            r = self.session.get(
                "https://api.openalex.org/works",
                params=params,
                timeout=15
            )
            self.last_request = time.time()

            if r.status_code == 200:
                results = r.json()["results"]

                if results:
                    # Check each result for similarity
                    best_match = None
                    best_score = 0

                    for paper in results:
                        score = self.similarity(title, paper["display_name"])
                        if score > best_score and score > 0.6:  # Lower threshold
                            best_score = score
                            best_match = paper

                    if best_match:
                        self.stats["success"] += 1
                        return {
                            "openalex_id": best_match.get("id"),
                            "publication_year": best_match.get("publication_year"),
                            "citation_count": best_match.get("cited_by_count"),
                            "referenced_works": best_match.get("referenced_works", []),
                            "title_matched": best_match.get("display_name"),
                            "similarity_score": best_score
                        }

                self.stats["not_found"] += 1
                return None

            elif r.status_code == 429:
                self.stats["rate_limited"] += 1
                print(f"⚠️ Rate limited. Waiting longer...")
                time.sleep(2)  # Simple backoff
                return self.search_paper(title, year)  # Retry

            else:
                self.stats["not_found"] += 1
                return None

        except Exception as e:
            print(f"❌ Error searching '{title[:50]}...': {e}")
            self.stats["not_found"] += 1
            return None

    def batch_search(self, papers_df, title_col="title", year_col=None,
                     save_every=100, checkpoint_file="openalex_matches.csv"):
        """
        Search multiple papers with checkpoints
        """
        results = []

        # Load existing checkpoint if available
        if os.path.exists(checkpoint_file):
            existing = pd.read_csv(checkpoint_file)
            results = existing.to_dict('records')
            print(f"📥 Loaded {len(results)} existing matches from checkpoint")
            processed_ids = set(existing.get('paper_ID', []))
        else:
            processed_ids = set()

        for idx, row in papers_df.iterrows():
            paper_id = row.get('paper_ID', idx)

            if paper_id in processed_ids:
                continue

            title = row[title_col]
            year = row.get(year_col) if year_col else None

            print(f"🔍 Searching {idx+1}/{len(papers_df)}: {title[:50]}...")

            match = self.search_paper(title, year)

            if match:
                results.append({
                    'paper_ID': paper_id,
                    'split': row.get('split'),
                    'domain': row.get('domain'),
                    'title': title,
                    **match
                })
            else:
                # Store even non-matches to avoid re-searching
                results.append({
                    'paper_ID': paper_id,
                    'split': row.get('split'),
                    'domain': row.get('domain'),
                    'title': title,
                    'openalex_id': None,
                    'publication_year': None,
                    'citation_count': None,
                    'referenced_works': None
                })

            # Save checkpoint periodically
            if (idx + 1) % save_every == 0:
                pd.DataFrame(results).to_csv(checkpoint_file, index=False)
                print(f"💾 Checkpoint saved: {len(results)} papers processed")
                print(f"📊 Stats: {self.stats}")

        # Final save
        pd.DataFrame(results).to_csv(checkpoint_file, index=False)
        print(f"✅ Batch complete! Final stats: {self.stats}")

        return pd.DataFrame(results)

# ============================================
# PART 3: Run the Search
# ============================================

print("\n🚀 Starting OpenAlex search...")
client = OpenAlexClient(email="your_email@gmail.com", delay=0.12)  # ~8 req/sec

# Search for all papers
paper_metadata = client.batch_search(
    paper_mapping,
    title_col="title",
    save_every=50,
    checkpoint_file="openalex_matches_checkpoint.csv"
)

print(f"\n📊 Final Results:")
print(f"   Total papers: {len(paper_metadata)}")
print(f"   Matched: {paper_metadata['openalex_id'].notna().sum()}")
print(f"   Match rate: {paper_metadata['openalex_id'].notna().mean():.1%}")

# ============================================
# PART 4: Build the Knowledge Graph
# ============================================

print("\n🏗️ Building Knowledge Graph...")

# Filter to matched papers only
matched_papers = paper_metadata[paper_metadata['openalex_id'].notna()].copy()

# 4.1 Create nodes.csv
nodes = []

# Paper nodes
for _, row in matched_papers.iterrows():
    node_id = f"P_{row['openalex_id'].split('/')[-1]}"  # Extract last part of OpenAlex ID
    nodes.append({
        'node_id': node_id,
        'node_type': 'Paper',
        'year': row['publication_year'],
        'domain': row['domain'],
        'split': row['split'],
        'name': None
    })

# Load triples to get entities
all_triples = []
for file in triplet_files:
    df = pd.read_csv(file)
    all_triples.append(df)
triples_df = pd.concat(all_triples, ignore_index=True)

# Filter triples for matched papers
matched_paper_ids = set(matched_papers['paper_ID'])
triples_df = triples_df[triples_df['paper_ID'].isin(matched_paper_ids)]

# Entity nodes
unique_entities = set()
for _, row in triples_df.iterrows():
    # Assuming the triple format is [paper_id, predicate, object]
    # Adjust based on your actual columns
    if len(row) >= 3:
        unique_entities.add(str(row[2]))  # object entity

for entity in unique_entities:
    entity_hash = hashlib.md5(entity.encode()).hexdigest()[:8]
    nodes.append({
        'node_id': f"E_{entity_hash}",
        'node_type': 'Entity',
        'year': None,
        'domain': None,
        'split': None,
        'name': entity
    })

nodes_df = pd.DataFrame(nodes)
nodes_df.to_csv('nodes.csv', index=False)
print(f"✅ Created nodes.csv with {len(nodes_df)} nodes")

# 4.2 Create knowledge_edges.csv
# Create entity lookup
entity_to_id = {}
for node in nodes:
    if node['node_type'] == 'Entity':
        entity_to_id[node['name']] = node['node_id']

knowledge_edges = []
for _, row in triples_df.iterrows():
    paper_id = row['paper_ID']
    # Find corresponding OpenAlex ID
    paper_row = matched_papers[matched_papers['paper_ID'] == paper_id].iloc[0]
    source_id = f"P_{paper_row['openalex_id'].split('/')[-1]}"

    # Get entity (assuming object is in column 2)
    entity = str(row[2])
    if entity in entity_to_id:
        knowledge_edges.append({
            'source': source_id,
            'target': entity_to_id[entity],
            'predicate': str(row[1]),  # predicate is column 1
            'year': paper_row['publication_year']
        })

knowledge_df = pd.DataFrame(knowledge_edges)
knowledge_df.to_csv('knowledge_edges.csv', index=False)
print(f"✅ Created knowledge_edges.csv with {len(knowledge_df)} edges")

# 4.3 Create citation_edges.csv
citation_edges = []
paper_id_to_node = {}

# Create paper ID mapping
for node in nodes:
    if node['node_type'] == 'Paper':
        paper_id_to_node[node['node_id']] = node

# Collect all citations
for _, row in matched_papers.iterrows():
    source_id = f"P_{row['openalex_id'].split('/')[-1]}"
    cited_works = row.get('referenced_works', [])

    if cited_works and isinstance(cited_works, list):
        for cited in cited_works:
            # Extract OpenAlex ID from URL
            cited_id = f"P_{cited.split('/')[-1]}"

            # Check if cited paper exists in our nodes
            if cited_id in paper_id_to_node:
                citation_edges.append({
                    'source': source_id,
                    'target': cited_id,
                    'year': row['publication_year']
                })
            else:
                # Create minimal node for cited paper
                nodes.append({
                    'node_id': cited_id,
                    'node_type': 'Paper',
                    'year': None,  # We don't know the year
                    'domain': None,
                    'split': None,
                    'name': None
                })
                paper_id_to_node[cited_id] = nodes[-1]

                citation_edges.append({
                    'source': source_id,
                    'target': cited_id,
                    'year': row['publication_year']
                })

citation_df = pd.DataFrame(citation_edges)
citation_df.to_csv('citation_edges.csv', index=False)
print(f"✅ Created citation_edges.csv with {len(citation_df)} edges")

# Update nodes.csv with new cited paper nodes
nodes_df = pd.DataFrame(nodes)
nodes_df.to_csv('nodes.csv', index=False)
print(f"✅ Updated nodes.csv with {len(nodes_df)} nodes")

# ============================================
# PART 5: Summary Statistics
# ============================================

print("\n📊 FINAL KG STATISTICS:")
print(f"   Total nodes: {len(nodes_df)}")
print(f"     - Paper nodes: {len(nodes_df[nodes_df['node_type'] == 'Paper'])}")
print(f"     - Entity nodes: {len(nodes_df[nodes_df['node_type'] == 'Entity'])}")
print(f"   Total knowledge edges: {len(knowledge_df)}")
print(f"   Total citation edges: {len(citation_df)}")
print(f"   Match rate with OpenAlex: {len(matched_papers)}/{len(paper_mapping)} = {len(matched_papers)/len(paper_mapping):.1%}")

📚 Loading SciND papers...
✅ Loaded 3964 papers
   Split distribution:
SKG      3279
NOVEL     601
BLOG       84
Name: split, dtype: int64
✅ Keeping 3880 scholarly papers

🚀 Starting OpenAlex search...
🔍 Searching 1/3880: Preview, Attend and Review Schema-Aware Curriculum...
🔍 Searching 2/3880: Constructing Multi-Modal Dialogue Dataset by Repla...
🔍 Searching 3/3880: Fine-grained Post-training for Improving Retrieval...
🔍 Searching 4/3880: Adding Chit-Chat to Enhance Task-Oriented Dialogue...
🔍 Searching 5/3880: Saying No is An Art Contextualized Fallback Respon...
🔍 Searching 6/3880: DIALKI Knowledge Identification in Conversational ...
🔍 Searching 7/3880: Contextual Rephrase Detection for Reducing Frictio...
🔍 Searching 8/3880: Reference-Centric Models for Grounded Collaborativ...
🔍 Searching 9/3880: Neural Path Hunter Reducing Hallucination in Dialo...
🔍 Searching 10/3880: Thinking Clearly, Talking Fast Concept-Guided Non-...
🔍 Searching 11/3880: Generation and Extraction Combined Di